# SynkroDMX — Spotify Scheduler (using module imports)

This notebook uses the **`synkrodmx.spotify` module** to:
- Authenticate with Spotify
- Build the minimal schedule (current track + following tracks in the queue if permitted)
- Launch a polling loop with callbacks

> Adjust the `SCOPE` according to your needs:  
> - Only current track: `user-read-currently-playing`  
> - Current track **and queue**: `user-read-currently-playing user-read-playback-state`


In [2]:
# Imports relativos para ejecutar el notebook desde notebooks/
import os
import sys

# Añade la raíz del repo al path
REPO_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if REPO_ROOT not in sys.path:
    sys.path.insert(0, REPO_ROOT)

errors = []

try:
    # Caso 1: paquete instalado o presente como synkrodmx.spotify
    from synkrodmx.spotify import (
        create_spotify_client,
        build_schedule,
        SchedulerPoller,
        PollerConfig,
    )
    print("✅ Import: synkrodmx.spotify")
except Exception as e:
    errors.append(("synkrodmx.spotify", repr(e)))
    try:
        # Caso 2: carpeta spotify/ en la raíz del repo
        from spotify import auth as _auth_mod
        from spotify import schedule as _schedule_mod
        from spotify import poller as _poller_mod
        from spotify import types as _types_mod

        create_spotify_client = _auth_mod.create_spotify_client
        build_schedule = _schedule_mod.build_schedule
        SchedulerPoller = _poller_mod.SchedulerPoller
        PollerConfig = _types_mod.PollerConfig
        print("✅ Import: spotify/ desde la raíz del repo")
    except Exception as e2:
        errors.append(("spotify", repr(e2)))
        try:
            # Caso 3: carpeta source/spotify/
            SOURCE_ROOT = os.path.join(REPO_ROOT, "source")
            if SOURCE_ROOT not in sys.path:
                sys.path.insert(0, SOURCE_ROOT)

            from spotify import auth as _auth_mod
            from spotify import schedule as _schedule_mod
            from spotify import poller as _poller_mod
            from spotify import types as _types_mod

            create_spotify_client = _auth_mod.create_spotify_client
            build_schedule = _schedule_mod.build_schedule
            SchedulerPoller = _poller_mod.SchedulerPoller
            PollerConfig = _types_mod.PollerConfig
            print("✅ Import: source/spotify/")
        except Exception as e3:
            errors.append(("source/spotify", repr(e3)))
            raise ImportError(
                "No se pudo importar el módulo Spotify. Intentos fallidos: "
                + " | ".join(f"{k}: {v}" for k, v in errors)
            )
# Autenticación con Spotify leyendo .env
SCOPE = "user-read-currently-playing user-read-playback-state"  # ajusta si no quieres cola
sp = create_spotify_client(scope=SCOPE)
print("Auth OK.")


✅ Import: source/spotify/
Auth OK.



# Snapshot of Playback Schedule

This cell retrieves the current playback schedule from Spotify by calling the `build_schedule` function with a limit of 5 queued tracks. It then prints a formatted JSON representation of the schedule if there is an active playback, or a message ("No hay reproducción activa") if no track is playing.


In [3]:

# Snapshot de la agenda (tema actual + siguientes si hay permiso)
import json
schedule = build_schedule(sp, max_queue_items=5)
print(json.dumps(schedule, ensure_ascii=False, indent=2) if schedule else "No hay reproducción activa.")


[
  {
    "track_id": "3SoDB59Y7dSZLSDBiNJ6o2",
    "uri": "spotify:track:3SoDB59Y7dSZLSDBiNJ6o2",
    "title": "Charlie",
    "artist": "Red Hot Chili Peppers",
    "artist_id": "0L8ExT028jH3ddEcZwqJJ5",
    "genres": [
      "funk rock",
      "alternative rock",
      "rock"
    ],
    "type": "track",
    "start_at_ms": 1756633584910,
    "end_at_ms": 1756633862443,
    "duration_ms": 277533,
    "context_uri": "spotify:album:2Y9IRtehByVkegoD7TcLfi",
    "is_current": true,
    "progress_ms": 126102,
    "timestamp_ms": 1756633711012,
    "colors": {
      "dominant_color": [
        [
          59,
          49,
          56
        ]
      ],
      "palette": [
        [
          59,
          49,
          56
        ],
        [
          124,
          153,
          189
        ],
        [
          211,
          185,
          39
        ],
        [
          86,
          102,
          144
        ],
        [
          144,
          168,
          148
        ]
     


# Plotly Visualization of Current Track

- Retrieves the current track schedule and extracts key details (artist, title, duration, current progress).
- Downloads the album cover image using a helper function; if unavailable, displays a placeholder.
- Constructs a two-panel figure:
    - **Left Panel:** Displays the album cover image.
    - **Right Panel:** Shows a horizontal timeline of the song's duration with a vertical line representing the current progress.
- Configures plot aesthetics such as titles, axis labels, and annotations to clearly present the playback information.


In [4]:
# %% Plotly display: artista, canción, carátula, timeline y paleta de 5 colores

import io
import time
import numpy as np
import requests
from PIL import Image
from colorthief import ColorThief
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Intentamos usar el helper del módulo; si no está, caemos a un helper local
try:
    from synkrodmx.spotify import get_album_cover_url  # expuesto por album_colors.py
except Exception:
    def get_album_cover_url(sp_client, track_id=None):
        if track_id:
            tr = sp_client.track(track_id)
            imgs = (tr.get("album") or {}).get("images") or []
            return imgs[0]["url"] if imgs else None
        cp = sp.current_user_playing_track()
        if cp and cp.get("item"):
            imgs = (cp["item"].get("album") or {}).get("images") or []
            return imgs[0]["url"] if imgs else None
        return None

def rgb_to_hex(rgb):
    r, g, b = [int(x) for x in rgb]
    return f"#{r:02x}{g:02x}{b:02x}"

# 1) Snapshot de agenda (solo el tema actual para ir rápido)
schedule = build_schedule(sp, max_queue_items=0, include_colors=False)
if not schedule:
    raise RuntimeError("No hay reproducción activa.")

cur = schedule[0]
artist = cur.get("artist") or ""
title = cur.get("title") or ""
duration_ms = int(cur.get("duration_ms") or 0)
progress_ms = int(cur.get("progress_ms") or 0)

duration_s = max(1e-6, duration_ms / 1000.0)
progress_s = max(0.0, min(duration_s, progress_ms / 1000.0))

# 2) Carátula del álbum y paleta (dominante + 4)
cover_url = get_album_cover_url(sp, track_id=cur.get("track_id"))
img_arr = None
swatches = []  # lista de RGB
if cover_url:
    try:
        resp = requests.get(cover_url, timeout=10)
        resp.raise_for_status()
        img_bytes = resp.content
        img = Image.open(io.BytesIO(img_bytes)).convert("RGB")
        img_arr = np.array(img)

        # Dominante + paleta con ColorThief
        thief = ColorThief(io.BytesIO(img_bytes))
        dominant = tuple(thief.get_color(quality=1))
        palette = thief.get_palette(color_count=5, quality=1) or []

        # Construimos vector de 5: dominante primero, luego hasta 4 de la paleta sin duplicar
        swatches = [dominant]
        for c in palette:
            c = tuple(c)
            if c not in swatches:
                swatches.append(c)
            if len(swatches) >= 5:
                break
    except Exception:
        img_arr = None
        swatches = []

# 3) Figura con tres columnas: carátula | timeline | paleta
fig = make_subplots(
    rows=1,
    cols=3,
    column_widths=[0.40, 0.45, 0.15],
    specs=[[{"type": "xy"}, {"type": "xy"}, {"type": "xy"}]],
    subplot_titles=("Album cover", "Timeline", "Palette (5)"),
)

# Carátula
if img_arr is not None:
    fig.add_trace(go.Image(z=img_arr), row=1, col=1)
else:
    fig.add_trace(
        go.Scatter(x=[0.5], y=[0.5], text=["No cover"], mode="text", hoverinfo="skip"),
        row=1, col=1,
    )
fig.update_xaxes(visible=False, row=1, col=1)
fig.update_yaxes(visible=False, row=1, col=1)

# Timeline: barra horizontal con línea vertical de progreso
fig.add_trace(
    go.Bar(
        x=[duration_s],
        y=["song"],
        orientation="h",
        hovertemplate="Duración: %{x:.1f}s<extra></extra>",
        showlegend=False,
    ),
    row=1, col=2,
)
# Línea de progreso
fig.add_shape(
    type="line",
    x0=progress_s, x1=progress_s,
    y0=-0.5, y1=0.5,
    xref="x2", yref="y2",
)

fig.update_yaxes(showticklabels=False, row=1, col=2)
fig.update_xaxes(title_text="seconds", range=[0, duration_s], row=1, col=2)

# Paleta: 5 parches verticales con su HEX centrado
if swatches:
    n = len(swatches)
    for i, rgb in enumerate(swatches):
        y0 = i / n
        y1 = (i + 1) / n
        hexcol = rgb_to_hex(rgb)
        fig.add_shape(
            type="rect",
            x0=0, x1=1, y0=y0, y1=y1,
            xref="x3", yref="y3",
            line=dict(width=1, color=hexcol),
            fillcolor=hexcol,
        )
    # Etiquetas HEX
    fig.add_trace(
        go.Scatter(
            x=[0.5] * n,
            y=[(i + 0.5) / n for i in range(n)],
            text=[rgb_to_hex(c) for c in swatches],
            mode="text",
            showlegend=False,
            hoverinfo="skip",
        ),
        row=1, col=3,
    )
else:
    fig.add_trace(
        go.Scatter(x=[0.5], y=[0.5], text=["No palette"], mode="text", hoverinfo="skip"),
        row=1, col=3,
    )
fig.update_xaxes(visible=False, range=[0, 1], row=1, col=3)
fig.update_yaxes(visible=False, range=[0, 1], row=1, col=3)

# Título con artista y canción + tiempo actual (sin guion largo)
fig.update_layout(
    title=f"{artist} - {title}   ({int(progress_s)}/{int(duration_s)} s)",
    height=520,
    margin=dict(l=40, r=40, t=60, b=40),
)

fig.show()


# Generate queue poll view
This cell sets up a continuous polling loop to monitor Spotify playback. It defines an on_change callback function that:
- Checks if a valid schedule is returned.
- Extracts and prints information about the currently playing track (including title, artist, and genres).
- Lists upcoming tracks if available.

A PollerConfig instance specifies the filename to save the last schedule snapshot. Then, a SchedulerPoller instance is created with the authenticated Spotify client, configuration, and callbacks for handling changes and errors. Finally, the poller starts its run_forever loop, continually checking for updates until interrupted (Ctrl+C).


In [ ]:
# Bucle de sondeo (Ctrl+C para detener)
def on_change(schedule):
    if not schedule:
        print("[poll] No hay reproducción activa.")
        return
    cur = schedule[0]
    artist = cur.get("artist") or ""
    genres = ", ".join((cur.get("genres") or [])[:3]) or "sin género"
    print(
        f"[event] {cur.get('title')} - {artist} [{genres}] "
        f"start={cur.get('start_at_ms')} end={cur.get('end_at_ms')}"
    )
    if len(schedule) > 1:
        nxt = " | ".join(
            f"{i+1}. {it.get('title')} - {it.get('artist')}"
            for i, it in enumerate(schedule[1:])
        )
        print("[event] Próximos:", nxt)

cfg = PollerConfig(save_last_schedule_json="last_schedule.json")
poller = SchedulerPoller(sp, cfg, on_change=on_change, on_error=lambda e: print(f"[error] {e}"))
print("Sondeando... Ctrl+C para parar.")
poller.run_forever()

Sondeando... Ctrl+C para parar.
[event] Right on Time - Red Hot Chili Peppers [funk rock, alternative rock, rock] start=1756633558566 end=1756633671099
[event] Próximos: 1. Road Trippin' - Red Hot Chili Peppers | 2. Fat Dance - 2006 Remaster - Red Hot Chili Peppers | 3. Over Funk - 2006 Remaster - Red Hot Chili Peppers | 4. Quixoticelixer - 2006 Remaster - Red Hot Chili Peppers | 5. Around the World - Red Hot Chili Peppers
[event] Right on Time - Red Hot Chili Peppers [funk rock, alternative rock, rock] start=1756633546087 end=1756633658620
[event] Próximos: 1. Road Trippin' - Red Hot Chili Peppers | 2. Fat Dance - 2006 Remaster - Red Hot Chili Peppers | 3. Over Funk - 2006 Remaster - Red Hot Chili Peppers | 4. Quixoticelixer - 2006 Remaster - Red Hot Chili Peppers | 5. Around the World - Red Hot Chili Peppers
[event] Right on Time - Red Hot Chili Peppers [funk rock, alternative rock, rock] start=1756633533457 end=1756633645990
[event] Próximos: 1. Road Trippin' - Red Hot Chili Peppers